In [1]:
# %%
import numpy as np
from scipy.spatial import Delaunay, minkowski_distance
import meshio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mesh import Mesh
import scipy.sparse.linalg as spla
from icosphere import icosphere

points, simplices = icosphere(30)
point_normals = points / np.linalg.norm(points,axis=-1,keepdims=True)
print(len(simplices))
mesh = Mesh(points, simplices, point_normals)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
18000


In [2]:
# %% Init
# \phi = x^2 + y^2 + (xy)^2
xc = mesh.barycenters[:,0]
yc = mesh.barycenters[:,1]
zc = mesh.barycenters[:,2]

phi = 1.0/np.cosh(100*(xc - 1.0))

xf = mesh.edge_centers[:,0]
yf = mesh.edge_centers[:,1]
zf = mesh.edge_centers[:,2]

#mass_flux_rate = (np.random.rand(*(len(xf),))-0.5)*2.0 # uf
mass_flux_rate = np.random.randn(*(len(xf),)) # uf
for _ in range(2):
    mass_flux_rate, _ = mesh.helmholtz_projection(mass_flux_rate)
mass_flux_rate /= np.max(np.abs(mass_flux_rate))
rhou_f = mass_flux_rate[:, np.newaxis] * mesh.edge_normals

J,rhs = mesh.convection_matrix(phi, rhou_f, face_flux=True)
Iv = mesh.identity_matrix()

Reset possion operator.
Reset convection operator.


In [3]:
with meshio.xdmf.TimeSeriesWriter("convection_test2.xdmf") as writer:
    writer.write_points_cells(points, [("triangle", simplices),])
    for t in range(1001):
        J,rhs = mesh.convection_matrix(phi, rhou_f, quick=True, face_flux=True)
        # Solve
        dt = 0.01
        crank_nicolson_coeff = 0.5
        phi = spla.gmres(Iv+crank_nicolson_coeff*dt*J, phi*mesh.areas-(1.0-crank_nicolson_coeff)*dt*J@phi+dt*rhs)[0]
        print(t*dt)
        if t % 10 == 0:
            writer.write_data(t*dt, cell_data={"phi": [phi]})

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36
0.37
0.38
0.39
0.4
0.41000000000000003
0.42
0.43
0.44
0.45
0.46
0.47000000000000003
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.96
0.97
0.98
0.99
1.0
1.01
1.02
1.03
1.04
1.05
1.06
1.07
1.08
1.09
1.1
1.11
1.12
1.1300000000000001
1.1400000000000001
1.1500000000000001
1.16
1.17
1.18
1.19
1.2
1.21
1.22
1.23
1.24
1.25
1.26
1.27
1.28
1.29
1.3
1.31
1.32
1.33
1.34
1.35
1.36
1.37
1.3800000000000001
1.3900000000000001
1.4000000000000001
1.41
1.42
1.43
1.44
1.45
1.46
1.47
1.48
1.49
1.5
1.51
1.52
1.53
1.54
1.55
1.56
1.